In [ ]:
# 导入所需的库
import wntr
import networkx as nx
import matplotlib.pyplot as plt

# 定义转换函数
def gpm_to_m3_per_s(gpm, is_us_gallon=True):
    if is_us_gallon:
        conversion_factor = 0.00378541  # US gallon to cubic meters
    else:
        conversion_factor = 0.00454609  # Imperial gallon to cubic meters
    return (gpm * conversion_factor) / 60.0

# 创建或加载管网模型
inp_file = 'networks/Net3.inp'  # 你可以使用自己的 .inp 文件路径
wn = wntr.network.WaterNetworkModel(inp_file)

# 使用 wntr 模拟管网
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

# 将 wntr 管网转换为 networkx 图
G = wn.get_graph()

# 可视化管网
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G)  # 选择布局算法
nx.draw_networkx_nodes(G, pos, node_size=50, node_color='blue')
nx.draw_networkx_edges(G, pos, edge_color='gray')
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')

# 添加节点标签（例如，节点名称）
node_labels = {node: node for node in G.nodes()}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=8, font_color='white')

# 显示图表
plt.title('管网拓扑图')
plt.axis('off')
plt.show()

# 分析结果
# 例如，获取某个时间点的压力数据
time = results.time[0]  # 选择第一个时间点
pressure_at_time = results.node['pressure'].loc[time, :]

# 打印压力数据
print(f"时间 {time} 的节点压力:")
print(pressure_at_time)

# 可视化压力分布
plt.figure(figsize=(10, 8))
node_colors = [pressure_at_time[node] for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=200, node_color=node_colors, cmap=plt.cm.coolwarm)
nx.draw_networkx_edges(G, pos, edge_color='gray')
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')

# 添加颜色条
sm = plt.cm.ScalarMappable(cmap=plt.cm.coolwarm, norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors)))
sm.set_array([])
plt.colorbar(sm, label='压力 (m)')

# 显示图表
plt.title(f'时间 {time} 的节点压力分布')
plt.axis('off')
plt.show()

# 流量单位转换示例
# 假设有一个节点的需求是 100 GPM
demand_gpm = 100
demand_m3_per_s = gpm_to_m3_per_s(demand_gpm)

print(f"需求 {demand_gpm} GPM 转换为 m³/s: {demand_m3_per_s:.6f} m³/s")